# `PromptTemplate + LLM`

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOllama(model="mistral")
chain = prompt | model

chain.invoke({"foo": "bears"})

AIMessage(content=' Why don\'t bears like to make left turns?\n\nBecause it\'s a "right bear" direction! (This one is a classic, feel free to share it with kids or those who enjoy a good pun!)')

## `Attaching Stop Sequences`

In [4]:
chain = prompt | model.bind(stop=["\n"])
chain.invoke({"foo": "bears"})

AIMessage(content=" Why don't bears like to make left turns?")

## `Attaching Function Call information`

In [5]:
from langchain_openai.chat_models import ChatOpenAI

functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {
                    "type": "string", 
                    "description": "The setup for the joke"
                },
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]

openai_model = ChatOpenAI()
chain = prompt | openai_model.bind(function_call={"name": "joke"}, functions=functions)
chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}', 'name': 'joke'}})

# `PromptTemplate + LLM + OutputParser`

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
chain.invoke({"foo": "bears"})

" Why don't bears like to make left turns?\n\nBecause they're right-paws!\n\n(I know, it's a pretty corny bear joke. But sometimes a good pun is all you need to bring a smile to your face!)"

## `Functions Output Parser`

In [7]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = prompt | openai_model.bind(function_call={"name": "joke"}, functions=functions) | JsonOutputFunctionsParser()
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [8]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | openai_model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke({"foo": "bears"})

"Why don't bears use cellphones?"

# `Simplifying input`

In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(foo=RunnablePassthrough())
chain = map_ | prompt | openai_model.bind(function_call={"name": "joke"}, functions=functions) | JsonOutputFunctionsParser()

chain.invoke("bears")

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}